# 词袋嵌入

我们通过**索引编码**把文字转换成数字，通过**单热编码**把数字转换成分类，从而解决了用**文字数据集**训练网络模型的第一步。

我们需要面对的第二个问题是：如何处理文字的**序列**（Sequence）。

文字是一种序列数据，我们不但要关心当前单词本身，还要考虑**上下文**（临近其他文字）的影响。所以我们不能单独研究一个单词，而是需要同时研究一句话，一段话，甚至一整篇文章。

一个直觉的方法是把一句话的索引编码作为一个向量来使用。

比如：可以把“我去上学”这句话的索引编码向量 [0, 8, 23, 48] 当作输入数据。这个向量不仅包括了一句话使用的单词，还包括每个单词的位置信息。

但是也有人会说：“我上学去”。索引编码向量是 [0, 23, 48, 8]。

这两句话意思相同，但是两个索引编码向量却只有 25% 相同。原因就是这种方法使用了文字的**绝对位置**信息。如果我们把使用**绝对位置**的向量，改成使用**相邻关系**的集合呢？那么这两句话的索引编码集合就都是 (0, 8, 23, 48)。

这种表达语序的方法，称为**词袋**（Bag of Words）。它的核心思路也很简单：把一句话使用到的单词，不分先后，放到一个集合中。

---

“我去上学”和“我上学去”这两句话的词袋是 (0, 8, 23, 48)。如果词表长度是 50 的话，对应的单热编码矩阵就是：

```data
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]
```

这还是词表长度只有 50 的情况。可以想象如果词表长度是数千、甚至数万的话，一句话的单热编码会是多么巨大。

对于稀疏向量的一种压缩方法是：**嵌入**（Embedding）。

它的核心思想并不复杂：每个单热编码向量只使用一个位置，其他都是 0。这种表达方式用于分类问题的标签值是有意义的；但是用于特征值却是巨大的浪费，同时会造成权重参数“爆表”。那么是否可以参考数据压缩技术，把每个单热编码向量压缩到一个相对较短的向量？

比如：把上面长度为 50 的单热编码向量压缩到长度为 8 或者 16 的向量；或者，把实际应用中长度几千，甚至几万的单热编码向量压缩到长度为 128 或者 256 的向量。

于是，**嵌入层**（Embedding Layer）应运而生。它的任务就是通过**词袋**和**嵌入**两种技术，把一段文字转换成适合网络模型处理的数据格式。

In [43]:
import csv
import math
import re
from abc import abstractmethod, ABC
from collections import Counter

import numpy as np

np.random.seed(99)

## 基础架构

### 张量

In [44]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return np.prod(self.data.shape[1:])

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [45]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

    def estimate(self, predictions):
        pass

### 基础层

In [46]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, x: Tensor):
        return self.forward(x)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [47]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [48]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

In [49]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### IMDB 数据集

在这个章节，我们将训练一个网络模型，可以根据观众影评的内容来预测观众态度。

观众态度只有两种：

* **1**：代表喜欢；
* **0**：代表讨厌。

网络模型的预测结果将是 [0, 1] 区间内的小数：

* **> 0.5**：预测观众喜欢；
* **< 0.5**：预测观众讨厌。

我们实现了基础数据集的**评估函数**（estimate），来评估网络模型预测的准确率。

In [50]:
class IMDBDataset(Dataset):

    def __init__(self, filename):
        self.filename = filename
        super().__init__()

    def load(self):
        self.reviews = []
        self.sentiments = []
        with open(self.filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for line in self.reviews:
            split_reviews.append(self.clean_text(line.lower()).split())

        self.vocabulary = set(word for line in split_reviews for word in line)
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[word] for word in line if word in self.word2index] for line in split_reviews]

    @staticmethod
    def clean_text(text):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text

    def train(self):
        self.features = [list(set(index)) for index in self.tokens[:-10]]
        self.labels = [0 if index == "negative" else 1 for index in self.sentiments[:-10]]

    def eval(self):
        self.features = [list(set(index)) for index in self.tokens[-10:]]
        self.labels = [0 if index == "negative" else 1 for index in self.sentiments[-10:]]

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def onehot(self, token):
        ebd = np.zeros(len(self.vocabulary))
        ebd[token] = 1
        return ebd

    @staticmethod
    def argmax(vector):
        return np.argmax(vector)

    def estimate(self, predictions):
        count = 0
        for i in range(len(predictions)):
            if np.abs(predictions[i].data - self.labels[i]) < 0.5:
                count += 1
        return count / len(predictions)

## 模型

### 线性层

In [51]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.randn(out_size, in_size) * np.sqrt(2 / in_size))
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 顺序层

In [52]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    @property
    def parameters(self):
        return [p for l in self.layers for p in l.parameters]

    def __repr__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 嵌入层

In [53]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=1):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        # 词表中的每个单词，都有自己的权重
        self.weight = Tensor(np.random.randn(embedding_size, vocabulary_size) * np.sqrt(2 / vocabulary_size))

    def forward(self, x: Tensor):
        # 取出词袋中每个单词的权重，并求和
        p = Tensor(np.sum(self.weight.data.T[x.data], axis=self.axis))

        def gradient_fn():
            if type(self.weight.grad) is not np.ndarray:
                self.weight.grad = np.zeros_like(self.weight.data)
            # 将梯度累加在词袋中每个单词对应的权重上
            self.weight.grad.T[x.data] += p.grad / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    @property
    def parameters(self):
        return [self.weight]

    def __repr__(self):
        return f'Embedding[weight{self.weight.data.shape}; vocabulary={self.vocabulary_size}; embedding={self.embedding_size}]'

### ReLU 激活函数

In [54]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += p.grad * (p.data > 0)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'ReLU[]'

### Sigmoid 激活函数

In [55]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad += p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Sigmoid[]'

### 损失函数（二元交叉熵）

In [56]:
class BCELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped) + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad += (clipped - y.data) / (clipped * (1 - clipped)) / len(y.data)

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

### 优化器（随机梯度下降）

In [57]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经网络模型

In [58]:
class WBModel(Model):

    def train(self, dataset, epochs):
        self.layer.train()
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                loss = self.loss_fn(predictions, labels)
                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()

    def test(self, dataset):
        self.layer.eval()
        dataset.eval()

        predictions = []
        for i in range(len(dataset)):
            feature, label = dataset[i]
            predictions.append(self.layer(feature))
        return predictions

## 设置

### 学习率

In [59]:
LEARNING_RATE = 0.01

### 轮次

In [60]:
EPOCHS = 100

## 训练

### 迭代

In [61]:
dataset = IMDBDataset('tinyimdb.csv')
layer = Sequential([Embedding(len(dataset.vocabulary), 64),
                    ReLU(),
                    Linear(64, 16),
                    ReLU(),
                    Linear(16, 1),
                    Sigmoid()])
loss = BCELoss()
optimizer = SGDOptimizer(layer.parameters, lr=LEARNING_RATE)

model = WBModel(layer, loss, optimizer)
model.train(dataset, EPOCHS)
print(layer)

Embedding[weight(64, 86); vocabulary=86; embedding=64]
ReLU[]
Linear[weight(16, 64); bias(16,)]
ReLU[]
Linear[weight(1, 16); bias(1,)]
Sigmoid[]


## 验证

### 测试

In [62]:
predictions = model.test(dataset)
print(f'Accuracy: {dataset.estimate(predictions)}')

Accuracy: 1.0


### 近义词

In [63]:
def similar(dataset, layer, target='excellent'):
    target_index = dataset.word2index[target]
    scores = Counter()

    for word, index in dataset.word2index.items():
        raw_diff = layer.layers[0].weight.data.T[index] - layer.layers[0].weight.data.T[target_index]
        squared_diff = raw_diff ** 2
        scores[word] = -math.sqrt(sum(squared_diff))

    return scores.most_common(10)

print(similar(dataset, layer, target='excellent'))
print(similar(dataset, layer, target='terrible'))

[('excellent', -0.0), ('these', -1.497579619272529), ('job', -1.585433058752462), ('and', -1.6465347632082759), ('on', -1.654060807478989), ('for', -1.7210414501354243), ('your', -1.7660287387179312), ('cinematography', -1.7769413657760007), ('liked', -1.7793144135106382), ('character', -1.7880578199719728)]
[('terrible', -0.0), ('actress', -1.5014821506491496), ('hated', -1.5448559584584955), ('in', -1.5839184945440659), ('us', -1.6082785987491568), ('poor', -1.609992375550338), ('soundtrack', -1.629265616709025), ('action', -1.6343208984418094), ('of', -1.6368476789459157), ('from', -1.6514544277466712)]
